## PREPARING THE DATA

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
mov = pd.read_csv("movies.csv")
mov.head()

d:\program files\python\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
mov.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

####  SELECT THE COLUMNS THAT CAN HELP IN RECOMMENDATION AND DISCARD THE REST

In [4]:
cleaned = mov[['id','title','original_language','genres','popularity','adult','vote_average','vote_count']]
cleaned.head()

,id,title,original_language,genres,popularity,adult,vote_average,vote_count
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.9469,False,7.7,5415.0
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.0155,False,6.9,2413.0
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",11.7129,False,6.5,92.0
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.85949,False,6.1,34.0
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]",8.38752,False,5.7,173.0


#### HANDLING MISSING DATA

In [5]:
cleaned.isnull().sum()

id                    0
title                 6
original_language    11
genres                0
popularity            5
adult                 0
vote_average          6
vote_count            6
dtype: int64

In [6]:
cleaned = cleaned.loc[cleaned["title"].notnull()]
cleaned.isnull().sum()

id                    0
title                 0
original_language    11
genres                0
popularity            0
adult                 0
vote_average          0
vote_count            0
dtype: int64

In [7]:
cleaned = cleaned.reset_index(drop=True)

In [8]:
cleaned.dtypes

id                    object
title                 object
original_language     object
genres                object
popularity            object
adult                 object
vote_average         float64
vote_count           float64
dtype: object

In [9]:
cleaned["popularity"]=cleaned["popularity"].astype(float)

#### HANDLING REDUNDANT MOVIES

In [10]:
cleaned.shape

(45460, 8)

In [11]:
len(cleaned["title"].unique())

42277

In [12]:
# we see that there are some redundant rows with movies

In [13]:
titles = dict()
for i in range(cleaned.shape[0]):
    try:
        titles[cleaned["title"][i]]+=1
    except:
        titles[cleaned["title"][i]] = 1

In [14]:
redundant = [x for x in titles.keys() if titles[x]>1]
red_movies = cleaned.loc[cleaned["title"].isin(redundant)]
red_movies = red_movies.reset_index(drop=True)
cleaned = cleaned.loc[~cleaned["title"].isin(redundant)]
cleaned = cleaned.reset_index(drop=True)

In [15]:
red_movies

,id,title,original_language,genres,popularity,adult,vote_average,vote_count
0,949,Heat,en,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",17.924927,False,7.7,1886.0
1,11860,Sabrina,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",6.677277,False,6.2,141.0
2,4584,Sense and Sensibility,en,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",10.673167,False,7.2,364.0
3,16420,Othello,en,"[{'id': 18, 'name': 'Drama'}]",1.845899,False,7.0,33.0
4,17015,Persuasion,en,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",2.228434,False,7.4,36.0
...,...,...,...,...,...,...,...,...
5508,52103,Pickpocket,zh,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",0.350365,False,7.2,13.0
5509,404604,Mom,hi,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",1.559596,False,6.6,14.0
5510,420346,The Morning After,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",0.139936,False,4.0,2.0
5511,30840,Robin Hood,en,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",5.683753,False,5.7,26.0


In [16]:
red_movies.loc[red_movies["title"]=="10 Minutes"]

,id,title,original_language,genres,popularity,adult,vote_average,vote_count
3273,118658,10 Minutes,bs,"[{'id': 10752, 'name': 'War'}]",0.283710,False,6.6,7.0
5249,244046,10 Minutes,ko,[],0.173285,False,7.5,2.0


In [17]:
rm = red_movies.groupby("title")    

In [18]:
def clean_redundant(group):
    cr = cleaned.iloc[0:0,:]
    group = group.reset_index(drop = True)
    title = group["title"][0]
    org_lang = group["original_language"][0]
    adult = group["adult"][0]
    id = group["id"][0]
    gen = list()
    pop = 0
    sum_vote = 0
    vot_avg = 0
    for i in range(group.shape[0]):
        gen.extend(group["genres"][i])
        pop = pop+ group["popularity"][i]*group["vote_count"][i]
        sum_vote = sum_vote + group["vote_count"][i]
        vot_avg = vot_avg + group["vote_average"][i]*group["vote_count"][i]
        
    pop = pop/sum_vote
    vot_avg = vot_avg/sum_vote
    gen = list(set(gen))
    clean = pd.DataFrame({'id':id,'title':title,'original_language':org_lang,'genres':gen,'popularity':pop,
                 'adult':adult,'vote_average':vot_avg,'vote_count':sum_vote})
    cleaned.append(clean)

In [19]:
for name,group in rm:
    clean_redundant(group)

d:\program files\python\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
d:\program files\python\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


In [20]:
cleaned.tail()

,id,title,original_language,genres,popularity,adult,vote_average,vote_count
39942,222848,Caged Heat 3000,en,"[{'id': 878, 'name': 'Science Fiction'}]",0.661558,False,3.5,1.0
39943,439050,Subdue,fa,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",0.072051,False,4.0,1.0
39944,111109,Century of Birthing,tl,"[{'id': 18, 'name': 'Drama'}]",0.178241,False,9.0,3.0
39945,227506,Satan Triumphant,en,[],0.003503,False,0.0,0.0
39946,461257,Queerama,en,[],0.163015,False,0.0,0.0


In [21]:
cleaned = cleaned.loc[cleaned["vote_count"]!=0]
cleaned.tail()
cleaned = cleaned.reset_index(drop = True)

#### EXTRACT GENRE FROM THE "GENRES" COLUMN AND MAKE SEPARATE COLUMN FOR EACH

In [22]:
all_genre = []
for i in range(cleaned.shape[0]):
    dict_gen = cleaned["genres"][i]
    dict_gen = dict_gen.replace("'",'"')
    dict_gen = json.loads(dict_gen)
    genre = []
    for j in dict_gen:
        genre.append(j["name"])
    all_genre.extend(genre)
    cleaned["genres"][i] = genre

d:\program files\python\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [23]:
cleaned.head()

,id,title,original_language,genres,popularity,adult,vote_average,vote_count
0,862,Toy Story,en,"[Animation, Comedy, Family]",21.946943,False,7.7,5415.0
1,8844,Jumanji,en,"[Adventure, Fantasy, Family]",17.015539,False,6.9,2413.0
2,15602,Grumpier Old Men,en,"[Romance, Comedy]",11.712900,False,6.5,92.0
3,31357,Waiting to Exhale,en,"[Comedy, Drama, Romance]",3.859495,False,6.1,34.0
4,11862,Father of the Bride Part II,en,[Comedy],8.387519,False,5.7,173.0


In [24]:
all_genre = set(all_genre)
#adding these to genres as columns to the dataframe
for col in all_genre:
    cleaned[col] = np.zeros(cleaned.shape[0])
    
# 1 if the genre exists in the movie genre list ... else 0
for i in range(cleaned.shape[0]):
    for col in all_genre:
        if col in cleaned["genres"][i]:
            cleaned[col][i] = 1
            
cleaned = cleaned.drop(columns = ['genres'])    
cleaned.head()

d:\program files\python\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,id,title,original_language,popularity,adult,vote_average,vote_count,Adventure,Crime,Action,...,War,Romance,Documentary,Horror,History,Animation,Foreign,Drama,Family,Mystery
0,862,Toy Story,en,21.946943,False,7.7,5415.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,8844,Jumanji,en,17.015539,False,6.9,2413.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,15602,Grumpier Old Men,en,11.712900,False,6.5,92.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31357,Waiting to Exhale,en,3.859495,False,6.1,34.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,11862,Father of the Bride Part II,en,8.387519,False,5.7,173.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### HANDLING CATEGORICAL VARIABLES

In [25]:
#converting the adult column in 1's and 0's
cleaned["adult"].unique()

array(['False', 'True'], dtype=object)

In [26]:
cleaned = cleaned.loc[(cleaned["adult"]=='False') | (cleaned["adult"]=='True')]
cleaned["adult"].unique()

array(['False', 'True'], dtype=object)

In [27]:
cleaned["adult"] = cleaned["adult"].replace(to_replace = ['False','True'],value = [0,1])
cleaned.head()

,id,title,original_language,popularity,adult,vote_average,vote_count,Adventure,Crime,Action,...,War,Romance,Documentary,Horror,History,Animation,Foreign,Drama,Family,Mystery
0,862,Toy Story,en,21.946943,0,7.7,5415.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,8844,Jumanji,en,17.015539,0,6.9,2413.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,15602,Grumpier Old Men,en,11.712900,0,6.5,92.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31357,Waiting to Exhale,en,3.859495,0,6.1,34.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,11862,Father of the Bride Part II,en,8.387519,0,5.7,173.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
cleaned["original_language"].unique()

array(['en', 'fr', 'zh', 'fa', 'nl', 'de', 'cn', 'it', 'ar', 'es', 'ru',
       'sv', 'ja', 'ko', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy', 'vi',
       'da', 'no', 'cs', 'nb', 'pl', 'el', 'sr', 'sh', 'xx', 'mk', 'ca',
       'fi', 'th', 'sk', 'hi', 'bs', 'tr', 'is', 'ps', 'ab', 'ka', 'mn',
       'bm', 'uk', 'af', 'bo', 'la', 'ku', 'lv', 'ta', 'sl', 'tl', 'ur',
       'rw', 'id', 'bg', 'mr', 'lt', 'et', 'kk', 'ms', 'sq', 'qu', 'te',
       'am', 'jv', 'tg', nan, 'ml', 'hr', 'lo', 'ay', 'kn', 'ne', 'pa',
       'gl', 'ky', 'sm', 'eu', 'hy', 'iu', 'si'], dtype=object)

In [29]:
cleaned.describe()

,popularity,adult,vote_average,vote_count,Adventure,Crime,Action,Western,Music,Science Fiction,...,War,Romance,Documentary,Horror,History,Animation,Foreign,Drama,Family,Mystery
count,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,...,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000,37293.000000
mean,3.010197,0.000188,6.001435,113.198295,0.078058,0.093771,0.148821,0.022551,0.035878,0.069665,...,0.029979,0.148580,0.087496,0.102834,0.031507,0.046711,0.036575,0.443274,0.063309,0.053898
std,5.902714,0.013699,1.298427,499.301391,0.268266,0.291514,0.355917,0.148469,0.185989,0.254584,...,0.170531,0.355679,0.282565,0.303746,0.174686,0.211022,0.187719,0.496778,0.243522,0.225818
min,0.000001,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.466254,0.000000,5.300000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.225055,0.000000,6.100000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.882092,0.000000,6.900000,36.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,547.488298,1.000000,10.000000,14075.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [30]:
# get dummy variables for all the languages
cleaned = pd.concat([cleaned,pd.get_dummies(cleaned["original_language"])],axis = 1)

In [31]:
cleaned.drop(columns = ["original_language"],inplace = True)
cleaned.head()

,id,title,popularity,adult,vote_average,vote_count,Adventure,Crime,Action,Western,...,tg,th,tl,tr,uk,ur,vi,wo,xx,zh
0,862,Toy Story,21.946943,0,7.7,5415.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,8844,Jumanji,17.015539,0,6.9,2413.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,15602,Grumpier Old Men,11.712900,0,6.5,92.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,31357,Waiting to Exhale,3.859495,0,6.1,34.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,11862,Father of the Bride Part II,8.387519,0,5.7,173.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


#### SAVE THE DATAFRAME FOR LATER USE

In [32]:
cleaned.to_csv("cleaned_movie.csv",index=False)